# 📊 SIGETI DWH - Analyse des KPI

Ce notebook permet d'analyser les données du Data Warehouse SIGETI via les 7 vues KPI métier.

In [ ]:
# Imports et configuration
import pandas as pd
import psycopg2
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Configuration de la connexion
DB_CONFIG = {
    'host': os.getenv('SIGETI_DB_HOST', 'host.docker.internal'),
    'port': os.getenv('SIGETI_DB_PORT', '5432'),
    'database': os.getenv('SIGETI_DB_NAME', 'sigeti_dwh'),
    'user': os.getenv('SIGETI_DB_USER', 'sigeti_user'),
    'password': os.getenv('SIGETI_DB_PASSWORD', 'sigeti123')
}

print("✅ Configuration chargée")

In [ ]:
# Fonction de connexion à la base
def get_connection():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("✅ Connexion à SIGETI DWH établie")
        return conn
    except Exception as e:
        print(f"❌ Erreur de connexion: {e}")
        return None

# Fonction pour exécuter une requête et retourner un DataFrame
def query_to_df(query, conn=None):
    if conn is None:
        conn = get_connection()
    
    if conn:
        try:
            df = pd.read_sql_query(query, conn)
            return df
        except Exception as e:
            print(f"❌ Erreur requête: {e}")
            return None
    return None

# Test de connexion
conn = get_connection()
if conn:
    conn.close()

## 🎯 KPI 1: Demandes par Statut

In [ ]:
# Analyse des demandes par statut
query_statut = "SELECT * FROM v_kpi_demandes_par_statut ORDER BY nb_demandes DESC;"
df_statut = query_to_df(query_statut)

if df_statut is not None:
    print("📊 Répartition des demandes par statut:")
    display(df_statut)
    
    # Graphique en secteurs
    fig = px.pie(df_statut, values='nb_demandes', names='statut', 
                 title='Répartition des Demandes par Statut')
    fig.show()

## 🗺️ KPI 2: Demandes par Type de Zone

In [ ]:
# Analyse par type de zone
query_type = "SELECT * FROM v_kpi_demandes_par_type;"
df_type = query_to_df(query_type)

if df_type is not None:
    print("🗺️ Répartition par type de zone:")
    display(df_type)
    
    # Graphique en barres
    fig = px.bar(df_type, x='nb_zone_industrielle', y='nb_hors_zone',
                 title='Demandes: Zone Industrielle vs Hors Zone',
                 labels={'nb_zone_industrielle': 'Zone Industrielle', 'nb_hors_zone': 'Hors Zone'})
    fig.show()

## 🏢 KPI 3: Performance par Entité

In [ ]:
# Analyse performance par entité
query_entite = "SELECT * FROM v_kpi_demandes_par_entite ORDER BY nb_demandes DESC LIMIT 10;"
df_entite = query_to_df(query_entite)

if df_entite is not None:
    print("🏢 Top 10 des entités par volume:")
    display(df_entite)
    
    # Graphique des top entités
    fig = px.bar(df_entite, x='entite_complete', y='nb_demandes',
                 title='Top 10 Entités par Nombre de Demandes')
    fig.update_xaxis(tickangle=45)
    fig.show()

## ⚡ KPI 4: Priorités vs Normales

In [ ]:
# Analyse priorités
query_priorite = "SELECT * FROM v_kpi_demandes_prioritaires;"
df_priorite = query_to_df(query_priorite)

if df_priorite is not None:
    print("⚡ Comparaison Prioritaires vs Normales:")
    display(df_priorite)
    
    # Graphique comparatif
    categories = ['Prioritaires', 'Normales']
    values = [df_priorite['nb_prioritaires'].iloc[0], df_priorite['nb_normales'].iloc[0]]
    
    fig = px.bar(x=categories, y=values, title='Demandes Prioritaires vs Normales',
                 color=categories, color_discrete_sequence=['red', 'blue'])
    fig.show()

## ⏱️ KPI 5: Délais de Traitement

In [ ]:
# Analyse des délais
query_delai = "SELECT * FROM v_kpi_delai_traitement;"
df_delai = query_to_df(query_delai)

if df_delai is not None:
    print("⏱️ Analyse des délais de traitement:")
    display(df_delai)
    
    # Graphique des tranches de délais
    if len(df_delai) > 0:
        tranches = ['≤7 jours', '8-15 jours', '>15 jours']
        values = [
            df_delai['nb_delai_court'].iloc[0],
            df_delai['nb_delai_moyen'].iloc[0], 
            df_delai['nb_delai_long'].iloc[0]
        ]
        
        fig = px.bar(x=tranches, y=values, title='Répartition des Délais de Traitement',
                     color=values, color_continuous_scale='RdYlGn_r')
        fig.show()

## ✅ KPI 6: Taux d'Acceptation

In [ ]:
# Analyse taux d'acceptation
query_acceptation = "SELECT * FROM v_kpi_taux_acceptation;"
df_acceptation = query_to_df(query_acceptation)

if df_acceptation is not None:
    print("✅ Taux d'acceptation:")
    display(df_acceptation)
    
    # Gauge du taux d'acceptation
    if len(df_acceptation) > 0:
        taux = df_acceptation['taux_acceptation'].iloc[0]
        
        fig = go.Figure(go.Indicator(
            mode = "gauge+number+delta",
            value = taux,
            domain = {'x': [0, 1], 'y': [0, 1]},
            title = {'text': "Taux d'Acceptation (%)"},
            gauge = {
                'axis': {'range': [None, 100]},
                'bar': {'color': "darkgreen"},
                'steps': [
                    {'range': [0, 50], 'color': "lightgray"},
                    {'range': [50, 85], 'color': "gray"}],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 90}}
        ))
        fig.show()

## 📈 KPI 7: Évolution Temporelle

In [ ]:
# Analyse évolution temporelle
query_evolution = "SELECT * FROM v_kpi_evolution_prioritaires ORDER BY mois;"
df_evolution = query_to_df(query_evolution)

if df_evolution is not None and len(df_evolution) > 0:
    print("📈 Évolution temporelle des priorités:")
    display(df_evolution)
    
    # Graphique d'évolution
    fig = px.line(df_evolution, x='mois', y='nb_prioritaires_mois',
                  title='Évolution Mensuelle des Demandes Prioritaires')
    fig.show()
else:
    print("ℹ️ Pas assez de données temporelles pour l'analyse d'évolution")

## 📊 Dashboard Synthèse

In [ ]:
# Dashboard de synthèse avec tous les KPI
print("📊 Dashboard de Synthèse SIGETI DWH")
print("=" * 40)

# Récupération de tous les KPI principaux
kpis = {
    'Demandes par Statut': query_to_df("SELECT COUNT(*) as total FROM v_kpi_demandes_par_statut"),
    'Types de Zone': query_to_df("SELECT * FROM v_kpi_demandes_par_type LIMIT 1"),
    'Entités Uniques': query_to_df("SELECT COUNT(DISTINCT entite_complete) as nb_entites FROM v_kpi_demandes_par_entite"),
    'Taux Acceptation': query_to_df("SELECT taux_acceptation FROM v_kpi_taux_acceptation LIMIT 1")
}

for nom, df in kpis.items():
    if df is not None and len(df) > 0:
        print(f"✅ {nom}: OK ({len(df)} ligne(s))")
    else:
        print(f"❌ {nom}: Aucune donnée")

print("\n🎯 Analyse terminée!")